In [42]:
import pandas as pd 
import numpy as np 
import re

In [12]:
df = pd.read_csv('SMSSpamCollection.txt', sep="\t", header=None)
df.rename(columns={0:'type', 1:'mail'}, inplace=True)
df.head()

,type,mail
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [14]:
dfspam = df.loc[df['type']=='spam']
dfspam

,type,mail
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
5,spam,FreeMsg Hey there darling it's been 3 week's n...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...
11,spam,"SIX chances to win CASH! From 100 to 20,000 po..."
...,...,...
5537,spam,Want explicit SEX in 30 secs? Ring 02073162414...
5540,spam,ASKED 3MOBILE IF 0870 CHATLINES INCLU IN FREE ...
5547,spam,Had your contract mobile 11 Mnths? Latest Moto...
5566,spam,REMINDER FROM O2: To get 2.50 pounds free call...


In [16]:
dfham = df.loc[df['type']=='ham']
dfham

,type,mail
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
6,ham,Even my brother is not like to speak with me. ...
...,...,...
5565,ham,Huh y lei...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [51]:
spam_word_list = dfspam['mail'].str.cat(sep = " ") #la combinaison de toutes les strings des mails
spam_word_list = pd.Series(list(set(re.split(r"[\W+$&+,:;=?@#|'<>.^*()%!-]",spam_word_list))))
spam_word_list 

0                     
1                  Ben
2       BCM1896WC1N3XX
3         icmb3cktz8r7
4               Tyrone
             ...      
3543             82468
3544             Lions
3545       09066368753
3546              comp
3547          MUNSTERS
Length: 3548, dtype: object

In [47]:

spam_word_list = dfspam['mail'].to_list()
spam_word_list = str(spam_word_list)
spam_word_list = spam_word_list.lower()
re.split(r"[\W+$&+,:;=?@#|'<>.^*()%!-]",spam_word_list)
spam_word_list = set(spam_word_list)
spam_word_list



{' ',
 '!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '<',
 '=',
 '>',
 '?',
 '@',
 '[',
 '\\',
 ']',
 '_',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '¡',
 '£',
 'ú',
 '’',
 '“'}

In [41]:
ham_word_list = dfham['mail'].to_list()
ham_word_list = str(ham_word_list)
ham_word_list = ham_word_list.lower()
ham_word_list = ham_word_list.split()
ham_word_list = set(ham_word_list)
ham_word_list

AttributeError: 'list' object has no attribute 'split'